In [6]:
# shark_tank_india_analysis.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

try:
    # Load the dataset
    print("Loading dataset...")
    df = pd.read_csv('ShartankIndiaAllPitches.csv')
    
    print("Dataset loaded successfully!")
    print("Dataset Shape:", df.shape)
    print("\nFirst look at the data:")
    print(df.head())
    
    print("\nColumn Information:")
    print(df.info())
    
    print("\nMissing Values:")
    print(df.isnull().sum())

    # DATA CLEANING STEPS

    # 1. Standardize Industry/Domain from 'Idea' column
    def categorize_industry(idea):
        if pd.isna(idea):
            return 'Other'
            
        idea_lower = str(idea).lower()
        
        if any(word in idea_lower for word in ['food', 'snack', 'restaurant', 'ice cream', 'sweets', 'beverage', 'kheer', 'pickle', 'momos', 'chips', 'lemonade', 'maggi']):
            return 'Food & Beverage'
        elif any(word in idea_lower for word in ['fashion', 'clothing', 'wear', 'shoes', 'underwear', 'sleeves', 'bags']):
            return 'Fashion & Apparel'
        elif any(word in idea_lower for word in ['tech', 'app', 'software', 'ai', 'automation', 'device', 'gadget', 'electric', 'smart', 'platform']):
            return 'Technology'
        elif any(word in idea_lower for word in ['health', 'wellness', 'medical', 'ayurvedic', 'fitness', 'hygiene', 'healthcare', 'dialysis']):
            return 'Health & Wellness'
        elif any(word in idea_lower for word in ['automobile', 'bike', 'scooter', 'vehicle', 'motorcycle', 'moto']):
            return 'Automotive & Mobility'
        elif any(word in idea_lower for word in ['toy', 'comic', 'entertainment', 'gifts', 'animation']):
            return 'Entertainment & Lifestyle'
        elif any(word in idea_lower for word in ['agriculture', 'farm', 'agro', 'hemp']):
            return 'Agriculture'
        elif any(word in idea_lower for word in ['eco', 'environment', 'sustainable', 'bamboo', 'green']):
            return 'Sustainability'
        else:
            return 'Other'

    print("\nCategorizing industries...")
    df['Industry'] = df['Idea'].apply(categorize_industry)

    # 2. Clean investment amounts and convert to numeric
    def clean_investment_amount(amount):
        if pd.isna(amount) or amount == 0 or amount == '0':
            return 0
        try:
            # Convert to string and remove any spaces
            amount_str = str(amount).strip()
            return float(amount_str)
        except:
            return 0

    print("Cleaning investment amounts...")
    df['Investment_Amount_Clean'] = df['Investment Amount (In Lakhs INR)'].apply(clean_investment_amount)
    df['Debt_Amount_Clean'] = df['Debt (In lakhs INR)'].apply(clean_investment_amount)

    # 3. Clean equity percentage
    def clean_equity(equity):
        if pd.isna(equity) or equity == 0 or equity == '0':
            return 0
        try:
            if isinstance(equity, str):
                equity = equity.replace('%', '').replace(',', '').strip()
            return float(equity)
        except:
            return 0

    print("Cleaning equity data...")
    df['Equity_Clean'] = df['Equity'].apply(clean_equity)

    # 4. Create Deal Success Flag
    df['Deal_Success'] = np.where(df['Investment_Amount_Clean'] > 0, 1, 0)

    # 5. Calculate total deal value
    df['Total_Deal_Value'] = df['Investment_Amount_Clean'] + df['Debt_Amount_Clean']

    # 6. Count number of investors per deal
    investor_columns = ['Anupam', 'Ashneer', 'Namita', 'Aman', 'Peyush', 'Vineeta', 'Ghazal']
    
    # Clean investor columns - handle NaN values
    for col in investor_columns:
        df[col] = df[col].fillna('N')
    
    df['Number_of_Investors'] = df[investor_columns].apply(lambda x: (x == 'Y').sum(), axis=1)

    # 7. Create investor participation columns
    for investor in investor_columns:
        df[f'{investor}_Invested'] = df[investor].apply(lambda x: 1 if x == 'Y' else 0)

    print("\nData Cleaning Completed!")
    print(f"Total Pitches: {len(df)}")
    print(f"Successful Deals: {df['Deal_Success'].sum()}")
    print(f"Success Rate: {(df['Deal_Success'].sum()/len(df)*100):.1f}%")

    # Save cleaned dataset
    df.to_csv('shark_tank_india_cleaned.csv', index=False)
    print("\nCleaned dataset saved as 'shark_tank_india_cleaned.csv'")

    # BASIC ANALYSIS
    print("\n" + "="*50)
    print("BASIC ANALYSIS RESULTS")
    print("="*50)

    # Success rate by industry
    industry_success = df.groupby('Industry').agg({
        'Deal_Success': ['count', 'sum', 'mean'],
        'Total_Deal_Value': 'sum',
        'Investment_Amount_Clean': 'mean'
    }).round(2)

    # Flatten column names
    industry_success.columns = ['Total_Pitches', 'Successful_Deals', 'Success_Rate', 'Total_Investment', 'Avg_Investment']
    industry_success = industry_success.sort_values('Success_Rate', ascending=False)

    print("\nSuccess Rate by Industry:")
    print(industry_success)

    # Investor analysis
    investor_stats = {}
    for investor in investor_columns:
        total_investments = df[f'{investor}_Invested'].sum()
        total_pitches = len(df)
        investment_rate = (total_investments / total_pitches * 100)
        investor_stats[investor] = {
            'Total_Investments': total_investments,
            'Investment_Rate': round(investment_rate, 1)
        }

    print("\nInvestor Participation Rates:")
    for investor, stats in investor_stats.items():
        print(f"{investor}: {stats['Total_Investments']} deals ({stats['Investment_Rate']}%)")

    # Create visualizations
    print("\nCreating visualizations...")
    plt.figure(figsize=(16, 12))

    # Plot 1: Success Rate by Industry
    plt.subplot(2, 2, 1)
    industry_success_sorted = industry_success.sort_values('Success_Rate', ascending=True)
    plt.barh(range(len(industry_success_sorted)), industry_success_sorted['Success_Rate'])
    plt.yticks(range(len(industry_success_sorted)), industry_success_sorted.index)
    plt.xlabel('Success Rate')
    plt.title('Deal Success Rate by Industry')
    plt.grid(axis='x', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(industry_success_sorted['Success_Rate']):
        plt.text(v + 0.01, i, f'{v:.2f}', va='center')

    # Plot 2: Total Investment by Industry
    plt.subplot(2, 2, 2)
    industry_investment = industry_success.sort_values('Total_Investment', ascending=True)
    plt.barh(range(len(industry_investment)), industry_investment['Total_Investment'])
    plt.yticks(range(len(industry_investment)), industry_investment.index)
    plt.xlabel('Total Investment (Lakhs INR)')
    plt.title('Total Investment by Industry')
    plt.grid(axis='x', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(industry_investment['Total_Investment']):
        plt.text(v + 5, i, f'{v:.0f}', va='center')

    # Plot 3: Investor Participation
    plt.subplot(2, 2, 3)
    investor_names = [inv for inv in investor_columns]
    investment_counts = [df[f'{inv}_Invested'].sum() for inv in investor_columns]
    bars = plt.bar(investor_names, investment_counts)
    plt.xticks(rotation=45)
    plt.ylabel('Number of Deals')
    plt.title('Investor Participation Count')
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, count in zip(bars, investment_counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                str(count), ha='center', va='bottom')

    # Plot 4: Deal Size Distribution
    plt.subplot(2, 2, 4)
    successful_deals = df[df['Deal_Success'] == 1]
    if len(successful_deals) > 0:
        plt.hist(successful_deals['Investment_Amount_Clean'], bins=15, edgecolor='black', alpha=0.7)
        plt.xlabel('Investment Amount (Lakhs INR)')
        plt.ylabel('Number of Deals')
        plt.title('Distribution of Successful Deal Sizes')
        plt.grid(alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'No successful deals', ha='center', va='center', transform=plt.gca().transAxes)

    plt.tight_layout()
    plt.savefig('shark_tank_analysis_charts.png', dpi=300, bbox_inches='tight')
    print("\nAnalysis charts saved as 'shark_tank_analysis_charts.png'")

    # Summary statistics
    print("\n" + "="*50)
    print("SUMMARY STATISTICS")
    print("="*50)
    print(f"Overall Success Rate: {df['Deal_Success'].mean()*100:.1f}%")
    
    if len(successful_deals) > 0:
        print(f"Average Investment (Successful Deals): {successful_deals['Investment_Amount_Clean'].mean():.1f} Lakhs INR")
        print(f"Average Equity Given: {successful_deals['Equity_Clean'].mean():.1f}%")
    else:
        print("No successful deals to calculate averages")
        
    print(f"Most Active Investor: {max(investor_stats, key=lambda x: investor_stats[x]['Total_Investments'])}")
    print(f"Industry with Highest Success Rate: {industry_success.index[0]}")
    
    # Additional insights
    print("\n" + "="*50)
    print("ADDITIONAL INSIGHTS")
    print("="*50)
    print(f"Total Investment Amount: {df['Investment_Amount_Clean'].sum():.0f} Lakhs INR")
    print(f"Number of Industries: {df['Industry'].nunique()}")
    print(f"Most Common Industry: {df['Industry'].mode().iloc[0]}")
    print(f"Average Investors per Successful Deal: {successful_deals['Number_of_Investors'].mean():.1f}")

    # Show sample of cleaned data
    print("\n" + "="*50)
    print("SAMPLE OF CLEANED DATA")
    print("="*50)
    sample_columns = ['Brand', 'Industry', 'Investment_Amount_Clean', 'Equity_Clean', 'Deal_Success', 'Number_of_Investors']
    print(df[sample_columns].head(10))

except Exception as e:
    print(f"Error occurred: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Make sure the CSV file is in the same directory as this script")
    print("2. Check that the file name is exactly 'ShartankIndiaAllPitches.csv'")
    print("3. Verify the CSV file is not corrupted")
    print("4. Ensure you have pandas installed: pip install pandas")
    
    # If there's an error, let's check what files are available
    import os
    print("\nFiles in current directory:")
    for file in os.listdir('.'):
        if file.endswith('.csv'):
            print(f"  - {file}")

print("\nScript execution completed!")

Loading dataset...
Dataset loaded successfully!
Dataset Shape: (117, 15)

First look at the data:
   Episode Number  Pitch Number                Brand  \
0               1             1  BluePine Industries   
1               1             2        Booz scooters   
2               1             3  Heart up my Sleeves   
3               2             4           Tagz Foods   
4               2             5       Head and Heart   

                                            Idea  \
0                                   Frozen Momos   
1  Renting e-bike for mobility in private spaces   
2                             Detachable Sleeves   
3                           Healthy Potato Chips   
4                       Brain Development Course   

   Investment Amount (In Lakhs INR)   Debt (In lakhs INR) Equity Anupam  \
0                                 75                    0    18%      N   
1                                 40                    0    50%      N   
2                          